In [ ]:
import skimage.io
import numpy as np
from matplotlib import pyplot as plt
import cv2 as cv
import time
# import imagecodecs
import imageio
from PIL import Image
from scipy import ndimage
import matplotlib.patches as patches # for drawing the rectangles on the field image
import pandas as pd

import os

# read image for potato June 22, 2020
# root_folder = "E:/data/remote_sensing/2020/"
# D:\data\remote_sensing\Potato_Fertilizer_Othello_Jun22_M10_transparenr_reflectance
root_folder = "c:/data/remote_sensing/"
project_stub = "Potato_Fertilizer_Othello_Jun22_M10_transparent_reflectance"

# export path for image files
export_path = os.path.join(root_folder, (project_stub + '/export/'))
  
# Create the directory if it doesn't already exist
try: 
    os.mkdir(export_path)
    print(f"Created: {export_path}")
except:
    print(f"Already exists: {export_path}")


# define float to int functions
def convert_float_to_int(file_path):
    


    image = cv.imread(file_path, cv.IMREAD_UNCHANGED)
    # print(np.min(float_image), np.max(float_image))
    # image = np.array(float_image * 255., dtype=np.uint8)
    # print(np.min(image), np.max(image))

    return image

# rotates an image, then returns a cropped image
def rotate_and_crop(img, rot_angle, y_limits, x_limits):
    # rotate then take the subset for our crop
    return ndimage.rotate(img, angle = rot_angle)[y_limits[0]:y_limits[1], x_limits[0]:x_limits[1]]

    

In [ ]:
# import all the wavelengths
blue = convert_float_to_int(root_folder + project_stub + "/" + project_stub + "_blue.tif")
blue_444 = convert_float_to_int(root_folder + project_stub + "/" + project_stub + "_blue-444.tif")
green = convert_float_to_int(root_folder + project_stub + "/" + project_stub + "_green.tif")
green_531 = convert_float_to_int(root_folder + project_stub + "/" + project_stub + "_green-531.tif")
red = convert_float_to_int(root_folder + project_stub + "/" + project_stub + "_red.tif")
red_650 = convert_float_to_int(root_folder + project_stub + "/" + project_stub + "_red-650.tif")
nir = convert_float_to_int(root_folder + project_stub + "/" + project_stub + "_nir.tif")
red_edge = convert_float_to_int(root_folder + project_stub + "/" + project_stub + "_red edge.tif")
red_edge_705 = convert_float_to_int(root_folder + project_stub + "/" + project_stub + "_red edge-705.tif")
red_edge_740 = convert_float_to_int(root_folder + project_stub + "/" + project_stub + "_red edge-740.tif")


    



In [ ]:
# show the uncropped image
rot_angle = 180
y_limits = [0, red.shape[0]]
x_limits = [0, red.shape[1]]

red_crop = rotate_and_crop(red, rot_angle, y_limits, x_limits)
nir_crop = rotate_and_crop(nir, rot_angle, y_limits, x_limits)

ndvi = (nir_crop - red_crop)/(nir_crop + red_crop)

ndvi_mask = np.where(ndvi>0.3, 1, 0)

# show the fig
plt.figure(figsize = (8,30))
plt.imshow(ndvi_mask, cmap="gray")
plt.savefig(export_path + 'uncropped_ndvi_mask.png')

In [ ]:
# definte the rotation angle and the desired field crop dimensions to isolate our region of interest
rot_angle = 182.4
y_limits = [2400, 9800]
x_limits = [1460, 3050]

# rotate and crop our red and nir bands
red_crop = rotate_and_crop(red, rot_angle, y_limits, x_limits)
nir_crop = rotate_and_crop(nir, rot_angle, y_limits, x_limits)

# calculate NDVI
ndvi = (nir_crop - red_crop)/(nir_crop + red_crop)

# use the NDVI image to create a mask
ndvi_mask = np.where(ndvi>0.45, 1, 0)

# # show NDVI
# plt.figure(figsize = (8,30))
# plt.imshow(ndvi, cmap="gray")
# plt.savefig('ndvi.png')

# show the mask
plt.figure(figsize = (8,30))
plt.imshow(ndvi_mask, cmap="gray")
plt.savefig(export_path + 'ndvi_mask.png')

In [ ]:
# # show the red data
# plt.figure(figsize = (8,30))
# plt.imshow(nir_crop, cmap="gray")
# plt.savefig(export_path + 'nir_crop.png')

In [ ]:
# create a hypercube of all the layers
channels = [blue, blue_444, green, green_531, red, red_650, nir, red_edge, red_edge_705, red_edge_740]

field_image = np.zeros(shape=(red_crop.shape[0], red_crop.shape[1], len(channels)))
print(field_image.shape)

# rotate all the images and crop them and create the field_image hypercube of just cropped/rot image channels
for c_idx, channel in enumerate(channels):
    
    channel = rotate_and_crop(channel, rot_angle, y_limits, x_limits)
#     print(f"channel.shape: {channel.shape}, ndvi_mask.shape: {ndvi_mask.shape}")
    
    
#     channel = np.multiply(channel, ndvi_mask)
    
    field_image[:,:,c_idx] = channel

bands_dict = {"blue": channels[0],
              "blue_444": channels[1],
              "green": channels[2],
              "green_531": channels[3],
              "red": channels[4],
              "red_650": channels[5],
              "nir": channels[6],
              "red_edge": channels[7],
              "red_edge_705": channels[8],
              "red_edge_740": channels[9]
             }
    

    
    
    
# # combine the channels into a single multilayerd image
# for i in range(0, len(channels)):
#     field_image[:,:,i] = rotate_and_crop(channels[i], rot_angle, y_limits, x_limits)
    
# import the plot map and ground truth data
ground_truth = pd.read_csv("C:/data/remote_sensing/Potato_Fertilizer_Othello_Jun22_M10_transparent_reflectance/2020-06-22_potato_yield_fertsplit.csv")

# create our plot index
plot_index = 0
df = pd.DataFrame(columns=['plot_id', 'origin', 'width', 'height'])

# create all our indices
# img_masked





# create channel dict
channel_dict = {'blue': 0, 'blue_444': 1, 'green': 2, 'green_531': 3, 'red': 4, 'red_650': 5, 'nir': 6, 'red_edge': 7, 'red_edge_705': 8, 'red_edge_740': 9}

# create the list that will hold the indices chosen
indices = []
index_name_list = []

# create list of indices
for i, v in channel_dict.items():
    current_channel = channel_dict[i]
    for j, b in channel_dict.items():
        index_combo = (current_channel, channel_dict[j])
        if index_combo not in indices and (index_combo[1] - index_combo[0] != 0):
            indices.append((current_channel, channel_dict[j]))
            index_name = "(" + i + "-" + j + ")/(" + i + "+" + j + ")"
            print(index_name)
            index_name_list.append(index_name)
print(len(indices))
img_indices = np.zeros(shape=(field_image.shape[0], field_image.shape[1], len(indices)))
print(img_indices.shape, "is the img_indices shape")
            
            
# # calculate the indices and store them in the img_indices
for i, idx in enumerate(indices):     
    # choose bands
    band1 = field_image[:, :, idx[0]]
    band2 = field_image[:, :, idx[1]]
    
    # calculate the index
    try: 
        print(idx[0], idx[1])
        current_index = np.divide(np.subtract(band1, band2), np.add(band1, band2))
    except:
        print(f"i {i} failed")
        current_index = np.zeros(shape=(band1.shape[0], band1.shape[1]))
    
    # mask the index
    img_indices[:,:, i] = np.multiply(current_index, ndvi_mask)
    img_indices[:,:, i] = current_index
    
print(f"The img_indices.shape is {img_indices.shape}")

In [ ]:
# show plot boundaries and do all the calculations for plot values in this cell

plot_height = 492
plot_width = 200
x_origins = [50, 355, 555, 850, 1050, 1350]
# x_origins = [1350, 1050, 850, 535, 335, 50] # RIGHT TO LEFT
y_origin = field_image.shape[0] - plot_height - 50

print(f"field_image.shape[0]: {field_image.shape[0]}, nir_crop.shape[0]: {nir_crop.shape[0]}")
edge_buf = 40

# list to keep a hold of the plot coordinates
plot_list = []

# define field image and subplots
figure, ax = plt.subplots(1, figsize=(8,30))

# Displays an image
ax.imshow(nir_crop, cmap="gray")
# ax.set_xlabel('time (s)')
ax.set_title('NDVI with plot boundaries\n' + project_stub)
plt.gca().legend(('plot border', 'plot region of interest'))
# ax.set_ylabel('Undamped')
i = 0
# add rectangle patches to image

# we create an instance of an object. This will be a dataframe, but there isn't anything to put in it yet.
df = None



# keep track of the plot ID with an index. We add one at the beginning of each loop, so we start with -1 so the first is 0
i = -1

# iterate through all the plot coordinates
for x_coord in x_origins:
    for range_y in range(0, 13):
        
        # increment i to keep track of plot ID
        i += 1
        
        # what is the current plot origin point
        current_point = (x_coord, y_origin - range_y * plot_height)
    
        # find the center of the plot for a text label
        plot_center = (current_point[0] + .5 * plot_width, current_point[1] + .5 * plot_height)
        
        # save these coordinates and plot id in a dict
        plot_dict = {"plot_id":i,
                    "origin":(int(current_point[0] + .5 * edge_buf), int(current_point[1] + 2 * edge_buf)),
                    "width":plot_width - edge_buf,
                    "height":plot_height - 4 * edge_buf}
        
        # add a rectangle showing the plot bounadies
        rect_line = patches.Rectangle((current_point[0], current_point[1]), plot_width, plot_height, edgecolor='r', lw=2, facecolor="r", alpha=.1)
        ax.add_patch(rect_line)

        # show the subsection of the plot that we are using
        rect_fill  = patches.Rectangle(xy=plot_dict["origin"],
                                        width=plot_dict["width"], 
                                        height=plot_dict["height"], 
                                        edgecolor='None', 
                                        facecolor="green", 
                                        alpha=.4)
        ax.add_patch(rect_fill)
        
        # show plot ID on the map
        ax.text(plot_center[0] - .3 * plot_width, plot_center[1] + .1 * plot_height, i, c='white')       
        ax.add_patch(rect_fill)
        
    
        # create roi coordinates for data slice
        x0 = plot_dict["origin"][0]
        x1 = plot_dict["origin"][0] + plot_dict["width"]
        y0 = plot_dict["origin"][1]
        y1 = plot_dict["origin"][1] + plot_dict["height"]

    
        # define the bands used for calculating our non-NDSI features
        # we use these to create our NDVI mask as well
#         channels = [blue, blue_444, green, green_531, red, red_650, nir, red_edge, red_edge_705, red_edge_740]
        nir1 = field_image[y0:y1, x0:x1, 6]
        red1 = field_image[y0:y1, x0:x1, 4]
        green1 = field_image[y0:y1, x0:x1, 2]
        blue1 = field_image[y0:y1, x0:x1, 0]
        
        # calculate the SIs
#         NDVI = np.divide(np.subtract(nir1, red1), np.add(nir1, red1))
        NDVI = (nir1 - red1) / (nir1 + red1)
        plt.imsave(export_path + "plot_" + str(i) + "_NDVI.jpeg", NDVI)
        
#         SAVI_num = np.multiply(1.5, np.subtract(nir1, red1))
#         SAVI_denom = np.add(np.add(nir1, red1), 0.5)
#         SAVI = np.divide(SAVI_num, SAVI_denom)
        SAVI = (1.5 * (nir1 + red1))/(0.5 + nir1 + red1)
        plt.imsave(export_path + "plot_" + str(i) + "_SAVI.jpeg", SAVI)
        
#         GCI = np.divide(nir1, np.subtract(green1, 1))
        GCI = (nir1)/(green1 - 1)
        plt.imsave(export_path + "plot_" + str(i) + "_GCI.jpeg", GCI, cmap='gray')
        #TGI = (((red1 - blue1)(red1 - green1)) - ((red1 - green1)(red1 - blue1))) / 2
        #TGI = np.multiply()
        
        
        
        # calculate NDVI_mask
        NDVI_mask = np.where(NDVI>.45, 1, 0) 
        plt.imsave(export_path + "plot_" + str(i) + "_NDVI_mask.jpeg", NDVI_mask)
        
        # get mean of masked index image and add to dict
        plot_dict["NDVI_mean"] = np.mean(np.multiply(NDVI, NDVI_mask))
        plot_dict["SAVI_mean"] = np.mean(np.multiply(SAVI, NDVI_mask))
        plot_dict["GCI_mean"] = np.mean(np.multiply(GCI, NDVI_mask))

        # iterate through all the layers in the img_indices and come up with ROI and values
        for index in range(img_indices.shape[2]):
            
            roi = img_indices[y0:y1, x0:x1, index]
#             print(f"i: {i}, index: {index}, roi.shape: {roi.shape}, NDVI_mask.shape: {NDVI_mask.shape}")
            roi_masked = np.multiply(roi, NDVI_mask)
            
            # get mean of masked index image and add to dict
            plot_dict[(str(index) + "_mean").strip()] = np.mean(roi_masked)
        
            # save an image of this ROI to disk
#             if i == 0:
#                 plt.imsave(export_path + "plot_" + str(i) + "_" + str(index) + ".jpeg", roi_masked)
            if index == 35:
                plt.imsave(export_path + "plot_" + str(i) + "_" + str(index) + ".jpeg", roi)
                
        
        # add the components to the dataframe
        if df is None:
            df = pd.DataFrame.from_dict(plot_dict)
        if df is not None:
            df = df.append(plot_dict,ignore_index=True)
            
        
        
        
        
#         channels = [blue, blue_444, green, green_531, red, red_650, nir, red_edge, red_edge_705, red_edge_740]
#         field_image = np.zeros(shape=(red_crop.shape[0], red_crop.shape[1], len(channels)))
        
            
#         roi = img_indices[y0:y1, x0:x1, index]
            
        
#             # add the index mean value into dictionary
#             plot_dict[(str(index) + "_mean").strip()] = np.mean(roi)
            

print(df.head())
print(df.tail())

# # now take that dictionary, turn it into a data frame
# df2 = pd.DataFrame.from_dict(my_dict)

    

# plt.savefig(export_path + "plot_map_boundaries.png")    

# print(f"expected 13 range x 6 rows = 78 total plots, actual plots: {i}")







# # take all the columns in df2 and add them into our original data frame
# for col in df2.columns:
#     df[col] = df2[col]
    
# # take a look at it
# print(df.head)

# # save it to disk
df.to_csv((export_path + 'hyperindices.csv').strip())

In [ ]:
# coords are y, x, z for shape
# slicing is y, x z as well
# origin is upper left

half_x = int(.5*img_indices.shape[1]) 
half_y = int(.5*img_indices.shape[0])
print(half_y, half_x)
temp_img = img_indices[0:half_y, 0:half_x, 35]
temp_mask = np.where(temp_img < -.45, 1, 0)
plt.imshow(temp_mask, cmap="gray")
print(temp_img.shape)


In [ ]:
plt.imshow(ndvi_mask, cmap="gray")
print(ndvi_mask.shape)

In [ ]:
# # show the red data
plt.figure(figsize = (8,30))
plt.imshow(NDVI_mask, cmap="gray")
# # plt.savefig(export_path + 'nir_crop.png')
# # # create list of indices
# # for i, v in channel_dict.items():
# #     current_channel = channel_dict[i]
# #     for j, b in channel_dict.items():
# #         index_combo = (current_channel, channel_dict[j])
# #         if index_combo not in indices and (index_combo[1] - index_combo[0] != 0):
# #             indices.append((current_channel, channel_dict[j]))
# #             index_name = "(" + i + " - " + j + ")/(" + i + " + " + j + ")"
# #             print(index_name)
# #             index_name_list.append(index_name)

In [ ]:

# my_dict = {'plot_id': df['plot_id']}

# # iterate through all the indices

# for i, index in enumerate(indices):
#     plot_id = []
#     mean_list = []
#     median_list = []
#     std_list = []
#     min_list = []
#     max_list = []
        
#     # iterate through the set of coordinates, and select the region of interest to calculate parameters
#     for j, coords in enumerate(df['coords']):
        
#         roi = img_indices[coords[0]:coords[0] + plot_height, coords[1]:coords[1]+plot_width, i]
           
#         # mean intensity
#         mean_list.append(np.mean(roi))
        
#         # median intensity
#         median_list.append(np.median(roi))
        
#         # standard deviation
#         std_list.append(np.std(roi)) 
        
#         # min intensity
#         min_list.append(np.min(roi))
        
#         # max intensity
#         max_list.append(np.max(roi))
        

#     # add the index lists into dictionary
#     my_dict[(str(i) + "_mean").strip()] = mean_list
#     my_dict[(str(i) + "_med").strip()] = median_list
#     my_dict[(str(i) + "_std").strip()] = std_list
#     my_dict[(str(i) + "_min").strip()] = min_list
#     my_dict[(str(i) + "_max").strip()] = max_list


# # now take that dictionary, turn it into a data frame
# df2 = pd.DataFrame.from_dict(my_dict)

# # take all the columns in df2 and add them into our original data frame
# for col in df2.columns:
#     df[col] = df2[col]
    
# # take a look at it
# print(df.head)

# # save it to disk
# df.to_csv((project_stub + '_hyperindices.csv').strip())

In [ ]:
# # # calculate all of the plot id and plot origin points, and add them to our data frame
# # for row in range(0, plot_row):
# #     for col in range(0, plot_col):
# #             # plot_id for merging later, get from plot map document
# #             plot_id = plot_map.iloc[row, col + 1]
# #             plot_index += 1
            
# #             # find the upper left origin
# #             current_point = (xy_begin[0] + row * plot_height, xy_begin[1] + col * plot_width)

# #             # add this to the dataframe
# #             df = df.append({'plot_id': plot_id,'coords': current_point},ignore_index=True)
    
# # # plot ndvi to check that our indices work and that our plots are correct
# # define field image and subplots
# img = img_indices[:,:,35]
# figure, ax = plt.subplots(1, figsize=(5,30))

# # Displays an image
# ax.imshow(img, cmap="gray")
# # ax.set_xlabel('time (s)')
# ax.set_title('NDVI with plot boundaries')

# # ax.set_ylabel('Undamped')

# # # add rectangle patches to image
# # for coord in df['coords']:
# #     rect = patches.Rectangle((coord[1],coord[0]),plot_width,plot_height, edgecolor='r', facecolor="none")
# #     ax.add_patch(rect)

   
    
# # plt.savefig(export_dir + "plot_map_boundaries.png")    

    
    
# # populate the data frame with some mean values of the plots at each index
# # TODO figure out the whole extraction and saving data thing
# #               # create the plot roi
# #             roi_crop = img_indices[current_point[0]:current_point[0] + plot_height, 
# #                                    current_point[1]:current_point[1]+plot_width, 
# #                                    :]
            
            
# # for i, index in enumerate(indices):
# #     mean_plot_value = np.mean(roi_crop[:,:,i])
# #     df = df.append({str(index): index,
# #                     (str(index)+"_mean"): mean_plot_value
# #                    }, ignore_index=True)

# # 







            
            

# # # # # merge our extracted features to the ground truth data
# # df = pd.merge(df,ground_truth,on='plot_id')
# # print(df.head)
# # # print("Time elapsed: ", time.time() - start_time)

In [ ]:

# my_dict = {'plot_id': df['plot_id']}

# # iterate through all the indices

# for i, index in enumerate(indices):
#     plot_id = []
#     mean_list = []
#     median_list = []
#     std_list = []
#     min_list = []
#     max_list = []
        
#     # iterate through the set of coordinates, and select the region of interest to calculate parameters
#     for j, coords in enumerate(df['coords']):
        
#         roi = img_indices[coords[0]:coords[0] + plot_height, coords[1]:coords[1]+plot_width, i]
           
#         # mean intensity
#         mean_list.append(np.mean(roi))
        
#         # median intensity
#         median_list.append(np.median(roi))
        
#         # standard deviation
#         std_list.append(np.std(roi)) 
        
#         # min intensity
#         min_list.append(np.min(roi))
        
#         # max intensity
#         max_list.append(np.max(roi))
        

#     # add the index lists into dictionary
#     my_dict[(str(i) + "_mean").strip()] = mean_list
#     my_dict[(str(i) + "_med").strip()] = median_list
#     my_dict[(str(i) + "_std").strip()] = std_list
#     my_dict[(str(i) + "_min").strip()] = min_list
#     my_dict[(str(i) + "_max").strip()] = max_list


# # now take that dictionary, turn it into a data frame
# df2 = pd.DataFrame.from_dict(my_dict)

# # take all the columns in df2 and add them into our original data frame
# for col in df2.columns:
#     df[col] = df2[col]
    
# # take a look at it
# print(df.head)

# # save it to disk
# df.to_csv((project_stub + '_hyperindices.csv').strip())

In [ ]:
print(red1)
print(NDVI)